In [1]:
import urllib.request
import json
import pymongo
import prov.model
import datetime
import uuid

import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


from datetime import datetime

#pd.__version__

%matplotlib inline

In [4]:
# exec(open('../pymongo_dm.py').read())

In [33]:
# Databas 1: Liquor Licenses
url = 'https://data.cityofboston.gov/resource/hda6-fnsh.json?liccat=CV7AL&city=Boston'
response = urllib.request.urlopen(url).read().decode("utf-8")
r = json.loads(response)
s = json.dumps(r, sort_keys=True, indent=2)

# repo.dropPermanent("liquor_license")
# repo.createPermanent("liquor_license")
# repo['enze.liquor_license'].insert_many(r)

df_liquor = pd.read_json(response)

addr = df_liquor['address'].apply(lambda d: d[1:].lower())

In [3]:
# Database 2: Crime Incident Reports
url = 'https://data.cityofboston.gov/resource/7cdf-6fgx.json?'
response = urllib.request.urlopen(url).read().decode("utf-8")
r = json.loads(response)
s = json.dumps(r, sort_keys=True, indent=2)

# repo.dropPermanent("crime_report")
# repo.createPermanent("crime_report")
# repo['enze.crime_report'].insert_many(r)

df_crime = pd.read_json(response)

In [4]:
# Database 3: Waze Jam Data
url = 'https://data.cityofboston.gov/resource/yqgx-2ktq.json?city=Boston,%20MA'
response = urllib.request.urlopen(url).read().decode("utf-8")
r = json.loads(response)
s = json.dumps(r, sort_keys=True, indent=2)

# repo.dropPermanent("waze_boston")
# repo.createPermanent("waze_boston")
# repo['enze.waze_boston'].insert_many(r)

df_waze = pd.read_json(response)

In [140]:
print(df_liquor.info())
# print(df_liquor.head(3))
print(df_crime.info())
# print(df_crime.head(3))
print(df_waze.info())
# print(df_waze.head(3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364 entries, 0 to 363
Data columns (total 23 columns):
address             364 non-null object
businessname        364 non-null object
capacity            364 non-null int64
city                364 non-null object
closing             364 non-null object
comments            290 non-null object
dbaname             364 non-null object
expdttm             364 non-null object
issdttm             364 non-null object
liccat              364 non-null object
liccatdesc          364 non-null object
licenseno           364 non-null object
licstatus           364 non-null object
location            364 non-null object
locationcomments    364 non-null object
opening             364 non-null object
patronsout          364 non-null object
phone               364 non-null object
primapplicant       364 non-null object
state               364 non-null object
stno                350 non-null float64
stnohi              70 non-null float64
zip            

In [ ]:
# MapReduce based on street.
# crime_totals = df_crime['streetname'].value_counts()

In [101]:
# df_crime[df_crime['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_mon_total = df_crime[df_crime.day_week == 'Monday']
df_crime_mon_total = df_crime_mon_total[df_crime_mon_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_tue_total = df_crime[df_crime.day_week == 'Tuesday']
df_crime_tue_total = df_crime_tue_total[df_crime_tue_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_wed_total = df_crime[df_crime.day_week == 'Wednesday']
df_crime_wed_total = df_crime_wed_total[df_crime_wed_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_thu_total = df_crime[df_crime.day_week == 'Thursday']
df_crime_thu_total = df_crime_thu_total[df_crime_thu_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_fri_total = df_crime[df_crime.day_week == 'Friday']
df_crime_fri_total = df_crime_fri_total[df_crime_fri_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_sat_total = df_crime[df_crime.day_week == 'Saturday']
df_crime_sat_total = df_crime_sat_total[df_crime_sat_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

df_crime_sun_total = df_crime[df_crime.day_week == 'Sunday']
df_crime_sun_total = df_crime_sun_total[df_crime_sun_total['streetname'].apply(lambda d: d.lower()).isin(addr)].streetname.value_counts()

In [102]:
crime_mon = df_crime_mon_total.to_frame(name='Monday')
# crime_total.reset_index(level=0, inplace=True)
# crime_total = crime_total.rename(columns = {'index':'Streetname'})

crime_tue = df_crime_tue_total.to_frame(name='Tuesday')
# crime_totals.reset_index(level=0, inplace=True)
# crime_totals = crime_totals.rename(columns = {'index':'Streetname'})

crime_wed = df_crime_wed_total.to_frame(name='Wednesday')

crime_thu = df_crime_thu_total.to_frame(name='Thursday')

crime_fri = df_crime_fri_total.to_frame(name='Friday')

crime_sat = df_crime_sat_total.to_frame(name='Saturday')

crime_sun = df_crime_sun_total.to_frame(name='Sunday')

In [142]:
# crime_total.merge(crime_totals, left_on='Monday', right_on='Tuesday', how='outer')

crime_totals = pd.concat([crime_mon,crime_tue,crime_wed,crime_thu,crime_fri,crime_sat,crime_sun],axis=1)

crime_totals

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
ATLANTIC AV,NaN,2,NaN,NaN,NaN,NaN,1
BOYLSTON ST,3,4,4,NaN,NaN,NaN,NaN
BROAD ST,NaN,NaN,NaN,NaN,NaN,NaN,1
BROOKLINE AV,2,NaN,1,NaN,NaN,NaN,NaN
CANAL ST,NaN,1,NaN,NaN,NaN,NaN,NaN
CHARLES ST,NaN,NaN,NaN,NaN,NaN,NaN,1
COMMERCIAL ST,NaN,1,NaN,NaN,NaN,NaN,1
COMMONWEALTH AV,7,6,5,NaN,NaN,NaN,1
DARTMOUTH ST,1,1,NaN,NaN,NaN,NaN,NaN
EXETER ST,NaN,NaN,1,NaN,NaN,NaN,NaN


In [121]:
# out_file = open("crime_totals.json","w")
# json.dump(crime_totals.to_json(),out_file, indent=4)

# jsondata = json.dumps(crime_totals.to_json())
# fd = open('/data/crime_totals.json', 'w')
# fd.write(jsondata)
# fd.close()

# import os

# filename = 'crime_totals.txt'
# dir = os.path.dirname(filename)
# if not os.path.exists(dir):
#     os.makedirs(dir)
# with open(filename, 'w'):
#     json.dump(crime_totals.to_json(),filename)

# with open("crime_totals.json", "w") as output_file:
#     json.dump(crime_totals.to_json(),output_file)

# print(json.dumps(crime_totals.to_json(), indent=4))

crime_totals.to_json()

'{"Monday":{"ATLANTIC AV":null,"BOYLSTON ST":3.0,"BROAD ST":null,"BROOKLINE AV":2.0,"CANAL ST":null,"CHARLES ST":null,"COMMERCIAL ST":null,"COMMONWEALTH AV":7.0,"DARTMOUTH ST":1.0,"EXETER ST":null,"FRANKLIN ST":null,"HANOVER ST":null,"HARRISON AV":5.0,"HUNTINGTON AV":3.0,"LAGRANGE ST":null,"LANSDOWNE ST":null,"LINCOLN ST":null,"MASSACHUSETTS AV":7.0,"MILK ST":null,"NEWBURY ST":5.0,"NORTHERN AV":null,"SCHOOL ST":null,"STANHOPE ST":null,"STATE ST":1.0,"SUMMER ST":2.0,"TREMONT ST":9.0,"WASHINGTON ST":12.0},"Tuesday":{"ATLANTIC AV":2.0,"BOYLSTON ST":4.0,"BROAD ST":null,"BROOKLINE AV":null,"CANAL ST":1.0,"CHARLES ST":null,"COMMERCIAL ST":1.0,"COMMONWEALTH AV":6.0,"DARTMOUTH ST":1.0,"EXETER ST":null,"FRANKLIN ST":null,"HANOVER ST":null,"HARRISON AV":2.0,"HUNTINGTON AV":4.0,"LAGRANGE ST":null,"LANSDOWNE ST":null,"LINCOLN ST":1.0,"MASSACHUSETTS AV":3.0,"MILK ST":1.0,"NEWBURY ST":null,"NORTHERN AV":2.0,"SCHOOL ST":null,"STANHOPE ST":2.0,"STATE ST":null,"SUMMER ST":null,"TREMONT ST":7.0,"WASHING

In [137]:
df_waze_total = df_waze[df_waze['street'].apply(lambda d: d.lower()).isin(addr)].street.value_counts()

waze_total = df_waze_total.to_frame(name='Waze')

waze_total

,Waze
Washington St,122
Tremont St,27
Hanover St,16
Boylston St,8
Newbury St,7
Summer St,6
Charles St,5
Dartmouth St,5
Commercial St,4
Lansdowne St,3


In [138]:
# out_file = open("waze_total.json","w")
# json.dump(waze_total.to_json(),out_file, indent=4)

waze_total.to_json()

'{"Waze":{"Washington St":122,"Tremont St":27,"Hanover St":16,"Boylston St":8,"Newbury St":7,"Summer St":6,"Charles St":5,"Dartmouth St":5,"Commercial St":4,"Lansdowne St":3,"Portland St":2,"Exeter St":2,"Lincoln St":2,"Arlington St":1}}'